##Prerequisites

In [0]:
df = spark.read.csv(
    path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/employee.csv",
    header=True,
    inferSchema=True,
    quote="'",
    sep="|"
)

df.display()

##handling missing Records

>Dropping Missing(NULL) Records

In [0]:
df.na.drop().display()

In [0]:
df.na.drop(subset=["name","id"]).display()

##Drop record as any column null

In [0]:
from pyspark.sql.functions import col
df.filter((col("name").isNotNull()) & (col("company").isNotNull())).display()

##Fill all null with Default values

In [0]:
df.na.fill("NULL_IN_SOURCE").na.fill(-1).display()

>Fill Null values with different values with diff columns

In [0]:
COLUMNS_WITH_DEFAULT={
  "id":-1,
  "name":"Anonymous",
  "exp":0
}

df.na.fill(COLUMNS_WITH_DEFAULT).display()

##Fill null values with Average

In [0]:
from pyspark.sql.functions import avg
avg_exp = df.select(avg("exp")).collect()[0][0]
COLUMN_WITH_DEFAULT= {
    "id":-1
    ,"name":'Anonymous'
    ,"exp":int(avg_exp)
    ,"doj":"1970-01-01"
}
df.na.fill(COLUMN_WITH_DEFAULT).display()

##Fill categorical Data with most Frequently (mode)

In [0]:
from pyspark.sql.functions import col
mode_gender=df.groupBy("gen").count().orderBy(col("count").desc()).first()[0]


print(mode_gender)
df.na.fill({
"id" :- 1,
"name": "Anonymous",
"exp":avg_exp,
"gen":mode_gender
}).display()

In [0]:
from pyspark.sql.functions import col,avg,when
avg_exp=df.filter(col("exp")>0).agg(avg("exp")).collect()[0][0]
print(avg_exp)

df.withColumn("exp", 
              when((col("exp")<0)|(col("exp").isNull()),avg_exp).otherwise(col("exp"))).display()

